# 店舗巡回ルート最適化アプリケーション

駅周辺の店舗を検索し、最適な巡回ルートを数理最適化で計算します。

## 使い方
1. セル1〜6を順番に実行して関数を定義
2. セル7のパラメータ（`STATION_NAME`, `KEYWORD`）を編集
3. セル7を実行して結果を取得
4. 表示されたGoogle Maps URLをクリックしてブラウザで地図を確認

## 変更点（2025-11-07）
- ipyleafletによるNotebook内地図表示をGoogle Maps URLリンクに変更（Kernelクラッシュ対策）
- Distance Matrix API の分割リクエストに対応（大量の店舗でも動作）
- 最適化ライブラリをmipからpython-tspに変更（Kernel安定性向上）

## セル1: ライブラリインポートと設定

In [ ]:
# ライブラリインポート
import os
from dotenv import load_dotenv
import googlemaps
import numpy as np
import pandas as pd
from python_tsp.exact import solve_tsp_dynamic_programming
from dataclasses import dataclass
from typing import List, Tuple, Optional
import webbrowser
import tempfile

# 環境変数の読み込み
load_dotenv()

# 設定
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
MAX_SEARCH_RESULTS = 5
TRAVEL_MODE = "walking"
WALKING_SPEED_KM_H = 4.0  # 徒歩速度 km/h

# APIキーの検証
if not GOOGLE_MAPS_API_KEY:
    raise ValueError("❌ GOOGLE_MAPS_API_KEYが設定されていません。.envファイルを確認してください。")

# Google Maps クライアント初期化
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

print("✅ 設定完了")
print(f"📍 最大検索結果数: {MAX_SEARCH_RESULTS}件")
print(f"🚶 移動手段: {TRAVEL_MODE}")

## セル2: データモデル定義

In [8]:
@dataclass
class Location:
    """位置情報を表すクラス"""
    lat: float  # 緯度
    lng: float  # 経度

    def __post_init__(self):
        if not (-90 <= self.lat <= 90):
            raise ValueError(f"緯度が範囲外です: {self.lat}")
        if not (-180 <= self.lng <= 180):
            raise ValueError(f"経度が範囲外です: {self.lng}")

    def to_tuple(self) -> Tuple[float, float]:
        return (self.lat, self.lng)


@dataclass
class Place:
    """店舗情報を表すクラス"""
    place_id: str
    name: str
    address: str
    location: Location
    distance_from_station: Optional[float] = None  # メートル

    def to_dict(self) -> dict:
        return {
            "店舗名": self.name,
            "住所": self.address,
            "駅からの距離": f"{self.distance_from_station:.0f}m" if self.distance_from_station else "N/A"
        }


@dataclass
class Station:
    """駅情報を表すクラス"""
    name: str
    location: Location

    def to_tuple(self) -> Tuple[float, float]:
        return self.location.to_tuple()


@dataclass
class Route:
    """最適化されたルート情報を表すクラス"""
    station: Station
    places: List[Place]  # 最適順序
    total_distance: float  # メートル
    segment_distances: List[float]  # 各区間の距離（メートル）

    @property
    def estimated_time(self) -> float:
        """推定所要時間（分）"""
        return (self.total_distance / 1000) / WALKING_SPEED_KM_H * 60

    def to_dataframe(self) -> pd.DataFrame:
        """ルート詳細のDataFrameを作成"""
        data = []
        cumulative_distance = 0

        for i, place in enumerate(self.places, 1):
            segment_dist = self.segment_distances[i-1]
            cumulative_distance += segment_dist
            data.append({
                "訪問順": i,
                "店舗名": place.name,
                "前地点からの距離": f"{segment_dist:.0f}m",
                "累積距離": f"{cumulative_distance:.0f}m"
            })

        return pd.DataFrame(data)

    def summary(self) -> pd.DataFrame:
        """サマリー情報のDataFrameを作成"""
        data = {
            "項目": ["総移動距離", "推定所要時間", "訪問店舗数"],
            "値": [
                f"{self.total_distance:.0f}m ({self.total_distance/1000:.2f}km)",
                f"{self.estimated_time:.1f}分",
                f"{len(self.places)}件"
            ]
        }
        return pd.DataFrame(data)

print("✅ データモデル定義完了")

✅ データモデル定義完了


## セル3: 店舗検索関数

In [9]:
def search_station(station_name: str) -> Station:
    """
    駅名から駅の位置情報を取得

    Args:
        station_name: 駅名

    Returns:
        Station: 駅情報オブジェクト

    Raises:
        ValueError: 駅が見つからない場合
    """
    try:
        result = gmaps.geocode(address=station_name, language="ja")

        if not result:
            raise ValueError(f"'{station_name}' に該当する駅が見つかりませんでした")

        location_data = result[0]["geometry"]["location"]
        location = Location(lat=location_data["lat"], lng=location_data["lng"])

        return Station(name=station_name, location=location)

    except Exception as e:
        if isinstance(e, ValueError):
            raise
        raise ValueError(f"駅の検索中にエラーが発生しました: {e}")


def search_nearby_places(station: Station, keyword: str, max_results: int = MAX_SEARCH_RESULTS) -> List[Place]:
    """
    駅周辺の店舗を検索

    Args:
        station: 駅情報
        keyword: 検索キーワード
        max_results: 最大取得件数

    Returns:
        List[Place]: 店舗リスト（駅から近い順）

    Raises:
        ValueError: 検索結果が0件の場合
    """
    try:
        result = gmaps.places_nearby(
            location=station.to_tuple(),
            keyword=keyword,
            rank_by="distance",
            language="ja"
        )

        if not result.get("results"):
            raise ValueError(f"'{keyword}' に該当する店舗が見つかりませんでした")

        places = []
        for place_data in result["results"][:max_results]:
            location_data = place_data["geometry"]["location"]
            location = Location(lat=location_data["lat"], lng=location_data["lng"])

            place = Place(
                place_id=place_data["place_id"],
                name=place_data["name"],
                address=place_data.get("vicinity", "住所情報なし"),
                location=location
            )
            places.append(place)

        if len(places) < max_results:
            print(f"⚠️ 検索結果が{len(places)}件のみでした")

        return places

    except Exception as e:
        if isinstance(e, ValueError):
            raise
        raise ValueError(f"店舗検索中にエラーが発生しました: {e}")


print("✅ 店舗検索関数定義完了")

✅ 店舗検索関数定義完了


## セル4: 距離計算関数

In [ ]:
def calculate_distance_matrix(station: Station, places: List[Place]) -> np.ndarray:
    """
    駅と全店舗間の距離行列を計算（分割リクエスト対応）

    距離行列の構造:
    - インデックス0: 駅
    - インデックス1〜N: 各店舗

    Args:
        station: 駅情報
        places: 店舗リスト

    Returns:
        np.ndarray: 距離行列 (N+1 x N+1)
    """
    # 全地点の位置情報リストを作成
    locations = [station.to_tuple()] + [place.location.to_tuple() for place in places]
    num_locations = len(locations)

    # 距離行列を初期化
    distance_matrix = np.zeros((num_locations, num_locations))

    # Distance Matrix APIの制限（1リクエストあたり25×25）
    BATCH_SIZE = 25

    try:
        # バッチ処理で距離行列を構築
        for i_start in range(0, num_locations, BATCH_SIZE):
            i_end = min(i_start + BATCH_SIZE, num_locations)
            origins_batch = locations[i_start:i_end]

            for j_start in range(0, num_locations, BATCH_SIZE):
                j_end = min(j_start + BATCH_SIZE, num_locations)
                destinations_batch = locations[j_start:j_end]

                # Distance Matrix APIを呼び出し
                result = gmaps.distance_matrix(
                    origins=origins_batch,
                    destinations=destinations_batch,
                    mode=TRAVEL_MODE,
                    language="ja",
                    units="metric"
                )

                # レスポンスから距離を抽出
                for i_local, i_global in enumerate(range(i_start, i_end)):
                    for j_local, j_global in enumerate(range(j_start, j_end)):
                        if i_global == j_global:
                            distance_matrix[i_global][j_global] = 0
                        else:
                            element = result["rows"][i_local]["elements"][j_local]
                            if element["status"] == "OK":
                                distance_matrix[i_global][j_global] = element["distance"]["value"]  # メートル
                            else:
                                # 経路が見つからない場合は大きな値を設定
                                distance_matrix[i_global][j_global] = 999999
                                print(f"⚠️ 地点{i_global}から地点{j_global}への経路が見つかりませんでした")

        # 店舗に駅からの距離を設定
        for idx, place in enumerate(places, 1):
            place.distance_from_station = distance_matrix[0][idx]

        print(f"📊 距離行列計算: {num_locations}地点 → {(num_locations + BATCH_SIZE - 1) // BATCH_SIZE}²回のAPIリクエスト")
        return distance_matrix

    except Exception as e:
        raise ValueError(f"距離計算中にエラーが発生しました: {e}")


print("✅ 距離計算関数定義完了")

## セル5: ルート最適化関数

In [ ]:
def optimize_route(station: Station, places: List[Place], distance_matrix: np.ndarray) -> Route:
    """
    TSPとして最適ルートを計算（python-tspライブラリ使用）

    Args:
        station: 駅情報
        places: 店舗リスト
        distance_matrix: 距離行列

    Returns:
        Route: 最適化されたルート情報
    """
    n = len(distance_matrix)  # 地点数（駅 + 店舗）

    # 距離行列を整数に変換（python-tspの要件）
    distance_matrix_int = distance_matrix.astype(int)

    # TSPを解く（動的計画法による厳密解）
    # permutation: 訪問順序のリスト（0から始まる）
    # distance: 総移動距離
    permutation, distance = solve_tsp_dynamic_programming(distance_matrix_int)

    # permutationを駅から始まるように調整
    # permutationは最適な巡回路を返すが、開始地点は任意
    # 駅（インデックス0）から始まるように回転させる
    station_idx = permutation.index(0)
    visit_order = permutation[station_idx:] + permutation[:station_idx]
    
    # 最後に駅に戻るように追加
    visit_order.append(0)

    # Routeオブジェクトを構築
    optimized_places = [places[i-1] for i in visit_order[1:-1]]  # 駅を除く

    # 各区間の距離を計算
    segment_distances = []
    for i in range(len(visit_order) - 1):
        segment_distances.append(distance_matrix[visit_order[i]][visit_order[i+1]])

    total_distance = sum(segment_distances)

    # 最後の店舗から駅への距離は除外（表示用）
    segment_distances_without_return = segment_distances[:-1]

    return Route(
        station=station,
        places=optimized_places,
        total_distance=total_distance,
        segment_distances=segment_distances_without_return
    )


print("✅ ルート最適化関数定義完了")

## セル6: 可視化関数

In [ ]:
def generate_google_maps_url(station: Station, places: List[Place], route_order: Optional[List[Place]] = None) -> str:
    """
    Google Maps URLを生成

    Args:
        station: 駅情報
        places: 店舗リスト
        route_order: ルート順序（最適化後の場合）

    Returns:
        str: Google Maps URL
    """
    # 起点（駅）
    origin = f"{station.location.lat},{station.location.lng}"
    
    if route_order:
        # 最適化ルートの場合：駅 → 店舗1 → ... → 店舗N → 駅
        waypoints = "|".join([f"{p.location.lat},{p.location.lng}" for p in route_order])
        destination = origin  # 駅に戻る
    else:
        # 検索結果表示の場合：単純に全店舗をウェイポイントとして表示
        if len(places) > 0:
            waypoints = "|".join([f"{p.location.lat},{p.location.lng}" for p in places[:-1]])
            destination = f"{places[-1].location.lat},{places[-1].location.lng}"
        else:
            waypoints = ""
            destination = origin
    
    # Google Maps Directions API URL構築
    base_url = "https://www.google.com/maps/dir/"
    
    if waypoints:
        url = f"{base_url}?api=1&origin={origin}&destination={destination}&waypoints={waypoints}&travelmode={TRAVEL_MODE}"
    else:
        url = f"{base_url}?api=1&origin={origin}&destination={destination}&travelmode={TRAVEL_MODE}"
    
    return url


def visualize_search_results(station: Station, places: List[Place]) -> str:
    """
    検索結果のGoogle Maps URLを生成して表示

    Args:
        station: 駅情報
        places: 店舗リスト

    Returns:
        str: Google Maps URL
    """
    url = generate_google_maps_url(station, places)
    print(f"🗺️ Google Mapsで開く: {url}")
    return url


def visualize_optimized_route(route: Route) -> str:
    """
    最適化されたルートのGoogle Maps URLを生成して表示

    Args:
        route: ルート情報

    Returns:
        str: Google Maps URL
    """
    url = generate_google_maps_url(route.station, route.places, route_order=route.places)
    print(f"🗺️ 最適ルートをGoogle Mapsで開く: {url}")
    return url


def display_search_results_table(places: List[Place]) -> pd.DataFrame:
    """
    検索結果を表形式で表示

    Args:
        places: 店舗リスト

    Returns:
        pd.DataFrame: 表示用DataFrame
    """
    data = []
    for idx, place in enumerate(places, 1):
        data.append({
            "番号": idx,
            "店舗名": place.name,
            "住所": place.address,
            "駅からの距離": f"{place.distance_from_station:.0f}m" if place.distance_from_station else "N/A"
        })
    return pd.DataFrame(data)


print("✅ 可視化関数定義完了")

## セル7: メイン実行

以下のパラメータを編集して実行してください。

In [ ]:
# ========================================
# パラメータ設定
# ========================================
STATION_NAME = "金山駅"      # 駅名を入力
KEYWORD = "手羽先"            # 検索キーワードを入力

print("="*60)
print(f"🚉 駅: {STATION_NAME}")
print(f"🔍 キーワード: {KEYWORD}")
print("="*60)

try:
    # ========================================
    # 1. 駅の検索
    # ========================================
    print("\n📍 駅を検索中...")
    station = search_station(STATION_NAME)
    print(f"✅ 駅の位置情報取得完了: {station.location.lat:.6f}, {station.location.lng:.6f}")

    # ========================================
    # 2. 店舗の検索
    # ========================================
    print(f"\n🏪 '{KEYWORD}' を検索中...")
    places = search_nearby_places(station, KEYWORD)
    print(f"✅ {len(places)}件の店舗を取得しました")

    # 検索結果を表示
    print("\n📊 検索結果（駅から近い順）:")
    df_search = display_search_results_table(places)
    display(df_search)

    # ========================================
    # 3. 距離行列の計算
    # ========================================
    print("\n📏 距離行列を計算中...")
    distance_matrix = calculate_distance_matrix(station, places)
    print(f"✅ 距離行列計算完了 ({len(distance_matrix)}x{len(distance_matrix)})")

    # 検索結果の地図URL
    print("\n🗺️ 検索結果の地図:")
    map_url_search = visualize_search_results(station, places)

    # ========================================
    # 4. ルートの最適化
    # ========================================
    print("\n🔧 最適ルートを計算中...")
    route = optimize_route(station, places, distance_matrix)
    print(f"✅ 最適化完了")

    # ルート詳細を表示
    print("\n📊 最適ルート詳細:")
    df_route = route.to_dataframe()
    display(df_route)

    print("\n📈 サマリー:")
    df_summary = route.summary()
    display(df_summary)

    print("\n🗺️ 最適ルートの地図:")
    map_url_route = visualize_optimized_route(route)

    print("\n" + "="*60)
    print("✅ すべての処理が完了しました！")
    print("="*60)
    print("\n💡 地図URLをクリックしてブラウザで開いてください")

except ValueError as e:
    print(f"\n❌ エラー: {e}")
    print("💡 入力内容を確認してください")
except Exception as e:
    print(f"\n❌ 予期しないエラー: {e}")
    print("💡 APIキーや設定を確認してください")